In [4]:
import torch
import tiktoken

In [5]:
with open( "datashort_story2.txt", "r" ) as f:
    raw_text = f.read()

raw_text[:50]

"'My Lord Chancellor,\n\n'When I consider the Affair "

In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
enc_text = tokenizer.encode(raw_text)

In [8]:
print(enc_text[:20])

[6, 3666, 4453, 19477, 11, 198, 198, 6, 2215, 314, 2074, 262, 6708, 958, 286, 281, 4479, 731, 86, 6346]


In [9]:
print( tokenizer.decode( enc_text[:2]))

'My


In [10]:
len( enc_text)

7407

In [11]:
for i in range(1,10):
    print("Input:", tokenizer.decode(enc_text[:i]), "Target:", tokenizer.decode([enc_text[i]]))

Input: ' Target: My
Input: 'My Target:  Lord
Input: 'My Lord Target:  Chancellor
Input: 'My Lord Chancellor Target: ,
Input: 'My Lord Chancellor, Target: 

Input: 'My Lord Chancellor,
 Target: 

Input: 'My Lord Chancellor,

 Target: '
Input: 'My Lord Chancellor,

' Target: When
Input: 'My Lord Chancellor,

'When Target:  I


In [12]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [13]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[    6,  3666,  4453, 19477],
        [   11,   198,   198,     6],
        [ 2215,   314,  2074,   262],
        [ 6708,   958,   286,   281],
        [ 4479,   731,    86,  6346],
        [  262,   734,  7973,    11],
        [  355,   340,   318,  6241],
        [  287,   262,  1811, 22698]])

Targets:
 tensor([[ 3666,  4453, 19477,    11],
        [  198,   198,     6,  2215],
        [  314,  2074,   262,  6708],
        [  958,   286,   281,  4479],
        [  731,    86,  6346,   262],
        [  734,  7973,    11,   355],
        [  340,   318,  6241,   287],
        [  262,  1811, 22698, 15370]])


In [15]:
# to apply the tokenizer's decoder to these IDs, the rows of the tensor `inputs` have to be converted into lists:
for row in inputs:
    print( tokenizer.decode( row.tolist() ) )

'My Lord Chancellor
,

'
When I consider the
 Affair of an
 Union betwixt
 the two Nations,
 as it is expressed
 in the several Articles


In [16]:
# we don't send these IDs to the LLM for training; we associate a vector a.k.a. tensor with each ID and then train the LLM on the vectors
# as a first example, let's create embedding vectors of length 3 for each token in a vocabulary of 6 tokens
vocab_size = 6
output_dim = 3
embedding = torch.nn.Embedding( vocab_size, output_dim )
print(embedding.weight)

Parameter containing:
tensor([[ 0.6212, -1.9226,  0.6053],
        [-0.9871,  1.4469, -0.2937],
        [-0.6023,  0.8544, -0.8292],
        [ 1.3791,  0.8162, -0.8097],
        [-0.2344, -0.4387, -0.3451],
        [ 0.4634,  0.2932,  1.0250]], requires_grad=True)


In [17]:
# if you just want the tensor part of this without the requires_grad=True bit
# method 1:
embedding.weight.data

tensor([[ 0.6212, -1.9226,  0.6053],
        [-0.9871,  1.4469, -0.2937],
        [-0.6023,  0.8544, -0.8292],
        [ 1.3791,  0.8162, -0.8097],
        [-0.2344, -0.4387, -0.3451],
        [ 0.4634,  0.2932,  1.0250]])

In [18]:
# if you just want the tensor part of this without the requires_grad=True bit
# method 1:
embedding.weight.detach()

tensor([[ 0.6212, -1.9226,  0.6053],
        [-0.9871,  1.4469, -0.2937],
        [-0.6023,  0.8544, -0.8292],
        [ 1.3791,  0.8162, -0.8097],
        [-0.2344, -0.4387, -0.3451],
        [ 0.4634,  0.2932,  1.0250]])

In [19]:
# call this A for some examples:
A = embedding.weight.detach()

In [20]:
# first row:
A[0]

tensor([ 0.6212, -1.9226,  0.6053])

In [21]:
# second row:
A[1]

tensor([-0.9871,  1.4469, -0.2937])

In [22]:
# first column:
A[:,0]

tensor([ 0.6212, -0.9871, -0.6023,  1.3791, -0.2344,  0.4634])

In [23]:
# element in row 2, column 3:
A[1,2]

tensor(-0.2937)

In [24]:
# to create a tensor directly:
x = torch.tensor([1.2,2.1])
y = torch.tensor([2.7,1.5])
print(x) 
print(y)

tensor([1.2000, 2.1000])
tensor([2.7000, 1.5000])


In [25]:
torch.dot( x,y)

tensor(6.3900)

In [26]:
# check:
1.2*2.7 + 2.1*1.5

6.390000000000001